Library

In [14]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd
import selenium
import matplotlib.pyplot as plt
JupyterDash._server_threads.clear()
import dash


Get data

In [15]:
file_path = "../../Data/Club/all_data_of_clubs.csv"
data = pd.read_csv(file_path)
data.rename(columns={"Unnamed: 0": "Team"}, inplace=True)
midfielder_data = pd.read_csv("../../Data/Stats_csv/Processed_Midfielder_data.csv")
forward_data = pd.read_csv("../../Data/Stats_csv/Processed_Forward_data.csv")

Prepare attacking data

In [16]:
attacking_features = ['Goals', 'Goals per match', 'Penalties scored', 'Shots', 'Shots on target', 'Big Chances Created', 'Shooting accuracy %']
attacking_data = data[['Team'] + attacking_features]

Prepare defensive data

In [17]:
defensive_features = ['Clean sheets', 'Goals Conceded', 'Tackles', 'Tackle success %', 'Errors leading to goal', 'Interceptions']
defensive_data = data[['Team'] + defensive_features]

Prepare prossessing data

In [18]:
control_features = ['Passes', 'Passes per match', 'Pass accuracy %', 'Crosses', 'Cross accuracy %']
control_data = data[['Team'] + control_features]

control_fig = px.bar(
    control_data.melt(id_vars='Team', value_vars=control_features, var_name='Metric', value_name='Value'),
    x='Team', y='Value', color='Metric',
    barmode='group',
    title="Comparison of Control Metrics Across Teams"
)

Heatmap of Correlations - Attacking Performance

In [19]:
attack_data_corr = data[['Goals', 'Goals per match', 'Penalties scored', 'Shots', 'Shots on target',
                         'Big Chances Created', 'Shooting accuracy %', 'Passes', 'Passes per match',
                         'Pass accuracy %']].corr()

heatmap_data = attack_data_corr.reset_index().melt(id_vars='index')
heatmap_data.columns = ['Feature1', 'Feature2', 'Correlation']

heatmap_fig = px.imshow(
    attack_data_corr,
    x=attack_data_corr.columns,
    y=attack_data_corr.columns,
    zmin=-1, zmax=1,
    color_continuous_scale='RdBu',
    title="Correlation Heatmap of Attack Metrics",
    labels={'color': 'Correlation'}
)

Heatmap of Correlations - Defensing Performance

In [20]:
defense_data_corr = data[['Clean sheets', 'Goals Conceded', 'Tackles', 'Tackle success %', 'Interceptions', 'Saves',
                          'Blocked shots', 'Clearances', 'Headed Clearance', 'Aerial Battles/Duels Won', 'Own goals',
                          'Passes', 'Passes per match', 'Pass accuracy %']].corr()

defense_heatmap_fig = px.imshow(
    defense_data_corr,
    x=defense_data_corr.columns,
    y=defense_data_corr.columns,
    zmin=-1, zmax=1,
    color_continuous_scale='RdBu',
    title="Correlation Heatmap of Defensive Metrics",
    labels={'color': 'Correlation'}
)

Top 10 best forward

In [21]:
finish_features = ['Name','Headed goals', 'Goals with right foot', 'Goals with left foot', 'Penalties scored', 'Freekicks scored']
forward_best_score = forward_data.sort_values(by='Goals', ascending=False).head(10)
forward_best_score_features = forward_best_score[finish_features]
fig_finish_features = px.bar(
    forward_best_score_features.melt(id_vars='Name', value_vars=finish_features, var_name='Metrics', value_name='Value'),
    x = 'Name',
    y = 'Value',
    color='Metrics',
    barmode='group',
    title='Comparison of Goal Scoring Features Among Players'
)

shot_features = ['Name', 'Shots', 'Shots on target', 'Shooting accuracy %', 'Hit woodwork', 'Big chances missed']
forward_shot_features = forward_best_score[shot_features]
fig_shot_features = px.bar(
    forward_shot_features.melt(id_vars='Name', value_vars=shot_features, var_name='Metrics', value_name='Value'),
    x = 'Name',
    y = 'Value',
    color = 'Metrics',
    barmode = 'group',
    color_discrete_sequence=['#b70ef1', '#0ed2f1', '#0ef148', '#f1c10e', '#f10e37']
)


Top 10 best midfielder

In [22]:
best_midfielder_data = midfielder_data.sort_values(by='Assists', ascending=False).head(10)
best_midfielder_data['Cross accuracy %'] = (
    best_midfielder_data['Cross accuracy %']
    .str.replace('%', '', regex=False)
    .astype(float)
)
available_features = ['Assists', 'Passes', 'Passes per match', 'Big Chances Created',
                      'Crosses', 'Cross accuracy %', 'Through balls', 'Accurate long balls']


Layout

In [23]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Football Clubs Performance Metrics", style={'textAlign': 'center'}),

    html.Div([
        html.H2("Attacking Metrics Trends", style={'textAlign': 'center'}),
        dcc.Graph(
            figure=px.line(
                attacking_data.melt(id_vars='Team', value_vars=attacking_features, var_name='Metric', value_name='Value'),
                x='Team', y='Value', color='Metric',
                title="Attacking Metrics Trends Across Teams"
            )
        )
    ]),

    html.Div([
        html.H2("Defensive Metrics Stacked Bar", style={'textAlign': 'center'}),
        dcc.Graph(
            figure=px.bar(
                defensive_data.melt(id_vars='Team', value_vars=defensive_features, var_name='Metric', value_name='Value'),
                x='Team', y='Value', color='Metric',
                title="Defensive Metrics Distribution Across Teams",
                barmode='stack'
            )
        )
    ]),

    html.Div([
        html.H2("Control Metrics Grouped Bar Chart", style={'textAlign': 'center'}),
        dcc.Graph(figure=control_fig)
    ]),

    html.Div([
        html.H2("Correlation Heatmap of Attack Metrics", style={'textAlign': 'center'}),
        dcc.Graph(figure=heatmap_fig)
    ]),
    html.Div([
        html.H2("Correlation Heatmap of Defensive Metrics", style={'textAlign': 'center'}),
        dcc.Graph(figure=defense_heatmap_fig)
    ]),


    html.Div([
        html.H2("Comparison of Goal Scoring Features Among Players", style={'textAlign': 'center'}),
        dcc.Graph(figure=fig_finish_features)
    ]),
    html.Div([
        html.H2("Comparison of Shot Features Among Players", style={'textAlign': 'center'}),
        dcc.Graph(figure=fig_shot_features)
    ]),

    html.Div([
        html.H2("Top 10 Midfielders: Feature Comparison", style={'textAlign': 'center'}),
        html.Div([
            html.Label("Select a Metric:", style={'fontSize': '18px'}),
            dcc.Dropdown(
                id='feature-dropdown',
                options=[{'label': feature, 'value': feature} for feature in available_features],
                value='Assists',
                style={'width': '50%', 'margin': 'auto'}
            )
        ], style={'textAlign': 'center', 'padding': '20px'}),
        html.Div([
            dcc.Graph(id='feature-comparison-chart')
        ])
    ])
])

@app.callback(
    dash.dependencies.Output('feature-comparison-chart', 'figure'),
    [dash.dependencies.Input('feature-dropdown', 'value')]
)
def update_feature_chart(selected_feature):
    fig = px.bar(
        best_midfielder_data,
        x='Name',
        y=selected_feature,
        title=f"Comparison of {selected_feature} for Top 10 Players",
        text=selected_feature
    )
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(yaxis_title='Value', xaxis_title='Player', template='plotly_white')
    return fig

app.run_server(debug=True)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
